In [2]:
import re
import sqlite3
from datetime import datetime
from pymongo import MongoClient
from dateutil.parser import parse

### Task 1: Extract Email Addresses and Dates

In [115]:
def extract_email_data(log_file):
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    date_pattern = r'(?:\w{3}\s+\d{1,2}\s+\d{2}:\d{2}:\d{2}\s+\d{4})|(?:\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})'
    
    email_data = []
    
    try:
        with open(log_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            
            for line in lines:
                # Find emails
                emails = re.findall(email_pattern, line)
                # Find dates
                dates = re.findall(date_pattern, line)
                
                if emails and dates:
                    for email in emails:
                        for date in dates:
                            email_data.append({'email': email, 'raw_date': date})
                            
    except FileNotFoundError:
        print(f"Error: File {log_file} not found")
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        
    return email_data

### Task 2: Data Transformation

In [118]:
def transform_data(email_data):
    transformed_data = []
    
    for item in email_data:
        try:
            # Parse and standardize date
            parsed_date = parse(item['raw_date'])
            formatted_date = parsed_date.strftime('%Y-%m-%d %H:%M:%S')
            
            transformed_data.append({
                'email': item['email'].lower(),
                'date': formatted_date
            })
        except ValueError as e:
            print(f"Error parsing date {item['raw_date']}: {str(e)}")
            
    return transformed_data

### Task 3: Save Data to MongoDB

In [121]:
def save_to_mongodb(data):
    try:
        client = MongoClient('mongodb://localhost:27017/')
        db = client['user_db']
        collection = db['user_history']
        
        # Insert data
        if data:
            collection.insert_many(data)
            print(f"Successfully inserted {len(data)} records to MongoDB")
            
        client.close()
    except Exception as e:
        print(f"Error saving to MongoDB: {str(e)}")

### Task 4: Database Connection and Data Upload

In [124]:
def save_to_sqlite(data):
    try:
        # Connect to SQLite
        conn = sqlite3.connect('user_history.db')
        cursor = conn.cursor()
        
        # Create table with constraints
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                email TEXT NOT NULL,
                date TEXT NOT NULL,
                UNIQUE(email, date)
            )
        ''')
        
        # Insert data
        for item in data:
            cursor.execute('''
                INSERT OR IGNORE INTO user_history (email, date)
                VALUES (?, ?)
            ''', (item['email'], item['date']))
        
        conn.commit()
        print(f"Successfully inserted records to SQLite")
        
    except sqlite3.Error as e:
        print(f"SQLite error: {str(e)}")
    finally:
        conn.close()

In [14]:
# Main execution
def main():
    log_file = 'C:/Guvi/mbox.txt'
    
    # Execute tasks
    email_data = extract_email_data(log_file)
    if not email_data:
        print("No data extracted")
        return
        
    transformed_data = transform_data(email_data)
    if not transformed_data:
        print("No data transformed")
        return
        
    save_to_mongodb(transformed_data)
    save_to_sqlite(transformed_data)

if __name__ == "__main__":
    main()

Successfully inserted 2140 records to MongoDB
Successfully inserted records to SQLite


### Queries:

In [131]:
import sqlite3

def run_query(query, params=()):
    try:
        conn = sqlite3.connect('user_history.db')
        cursor = conn.cursor()
        cursor.execute(query, params)
        results = cursor.fetchall()
        conn.close()
        return results
    except sqlite3.Error as e:
        print(f"SQLite error: {str(e)}")
        return []

def print_results(query, results):
    for row in results:
        print(row)

#### List all unique email addresses.

In [134]:
query = "SELECT DISTINCT email FROM user_history ORDER BY email;"
results = run_query(query)
print_results(query, results)

('a.fish@lancaster.ac.uk',)
('aaronz@vt.edu',)
('ajpoland@iupui.edu',)
('antranig@caret.cam.ac.uk',)
('arwhyte@umich.edu',)
('bahollad@indiana.edu',)
('bkirschn@umich.edu',)
('chmaurer@iupui.edu',)
('colin.clark@utoronto.ca',)
('csev@umich.edu',)
('cwen@iupui.edu',)
('david.horwitz@uct.ac.za',)
('dlhaines@umich.edu',)
('dsobiera@indiana.edu',)
('eli@media.berkeley.edu',)
('gbhatnag@umich.edu',)
('ggolden@umich.edu',)
('gjthomas@iupui.edu',)
('gopal.ramasammycook@gmail.com',)
('gsilver@umich.edu',)
('hu2@iupui.edu',)
('ian@caret.cam.ac.uk',)
('jbush@rsmart.com',)
('jholtzman@berkeley.edu',)
('jimeng@umich.edu',)
('jleasia@umich.edu',)
('jlrenfro@ucdavis.edu',)
('john.ellis@rsmart.com',)
('joshua.ryan@asu.edu',)
('josrodri@iupui.edu',)
('jzaremba@unicon.net',)
('kimsooil@bu.edu',)
('knoop@umich.edu',)
('ktsao@stanford.edu',)
('lance@indiana.edu',)
('louis@media.berkeley.edu',)
('mbreuker@loi.nl',)
('mmmay@indiana.edu',)
('nuno@ufp.pt',)
('ostermmg@whitman.edu',)
('ray@media.berkeley.edu'

#### Count the number of emails received per day.

In [137]:
query = """
SELECT DATE(date) AS email_date, COUNT(*) AS email_count
FROM user_history
GROUP BY DATE(date)
ORDER BY email_date;
"""
results = run_query(query)
print_results(query, results)

('2007-04-17', 1)
('2007-06-12', 1)
('2007-06-18', 1)
('2007-06-19', 5)
('2007-06-26', 2)
('2007-06-27', 2)
('2007-07-02', 1)
('2007-07-16', 1)
('2007-07-17', 1)
('2007-07-27', 1)
('2007-08-11', 1)
('2007-08-20', 1)
('2007-08-21', 1)
('2007-08-24', 3)
('2007-08-30', 3)
('2007-09-12', 1)
('2007-09-21', 1)
('2007-09-26', 1)
('2007-10-03', 1)
('2007-10-05', 2)
('2007-10-06', 7)
('2007-10-08', 1)
('2007-10-09', 2)
('2007-10-10', 2)
('2007-10-12', 2)
('2007-10-15', 2)
('2007-10-16', 1)
('2007-10-18', 14)
('2007-10-19', 29)
('2007-10-22', 16)
('2007-10-23', 36)
('2007-10-24', 26)
('2007-10-25', 40)
('2007-10-26', 39)
('2007-10-28', 2)
('2007-10-29', 100)
('2007-10-30', 62)
('2007-10-31', 24)
('2007-11-01', 25)
('2007-11-02', 18)
('2007-11-04', 27)
('2007-11-05', 37)
('2007-11-06', 77)
('2007-11-07', 34)
('2007-11-08', 33)
('2007-11-09', 29)
('2007-11-10', 17)
('2007-11-11', 5)
('2007-11-12', 21)
('2007-11-13', 26)
('2007-11-14', 48)
('2007-11-15', 41)
('2007-11-16', 27)
('2007-11-18', 2)
('2

#### Find the first and last email date for each email address.

In [140]:
query = """
SELECT email, MIN(date) AS first_email, MAX(date) AS last_email
FROM user_history
GROUP BY email
ORDER BY email;
"""
results = run_query(query)
print_results(query, results)

('a.fish@lancaster.ac.uk', '2007-10-29 08:00:00', '2007-12-19 10:34:30')
('aaronz@vt.edu', '2007-10-18 13:14:54', '2008-01-02 09:13:10')
('ajpoland@iupui.edu', '2007-10-19 09:24:52', '2007-12-18 15:46:20')
('antranig@caret.cam.ac.uk', '2007-11-06 11:34:04', '2008-01-04 10:04:14')
('arwhyte@umich.edu', '2007-11-06 15:49:36', '2007-12-20 11:58:05')
('bahollad@indiana.edu', '2007-10-26 08:51:00', '2007-11-05 13:13:07')
('bkirschn@umich.edu', '2007-06-18 17:12:48', '2007-12-26 16:33:12')
('chmaurer@iupui.edu', '2007-10-18 11:51:45', '2008-01-02 16:55:17')
('colin.clark@utoronto.ca', '2007-10-26 19:22:56', '2007-10-26 19:25:59')
('csev@umich.edu', '2007-11-05 10:49:26', '2007-12-30 22:44:30')
('cwen@iupui.edu', '2007-10-19 15:30:58', '2008-01-04 11:37:30')
('david.horwitz@uct.ac.za', '2007-09-26 05:26:53', '2008-01-04 13:05:51')
('dlhaines@umich.edu', '2007-10-18 10:56:16', '2007-12-21 13:56:39')
('dsobiera@indiana.edu', '2007-10-24 15:54:09', '2007-11-16 16:08:33')
('eli@media.berkeley.edu

#### Count the total number of emails from each domain.

In [143]:
query = """
SELECT SUBSTR(email, INSTR(email, '@') + 1) AS domain, COUNT(*) AS email_count
FROM user_history
GROUP BY domain
ORDER BY email_count DESC;
"""
results = run_query(query)
print_results(results)

TypeError: print_results() missing 1 required positional argument: 'results'

#### Question_1: How many emails were received each hour of the day (across all days)?

In [146]:
query = """
SELECT STRFTIME('%H', date) AS hour_of_day, COUNT(*) AS email_count
FROM user_history
GROUP BY hour_of_day
ORDER BY hour_of_day;
"""
print_results(query, run_query(query))

('00', 24)
('01', 13)
('02', 17)
('03', 21)
('04', 33)
('05', 14)
('06', 47)
('07', 45)
('08', 80)
('09', 184)
('10', 224)
('11', 180)
('12', 130)
('13', 133)
('14', 185)
('15', 188)
('16', 202)
('17', 112)
('18', 64)
('19', 56)
('20', 41)
('21', 46)
('22', 29)
('23', 23)


#### Question_2: Which email addresses sent more than one email?

In [149]:
query = """
SELECT email, COUNT(*) AS email_count
FROM user_history
GROUP BY email
HAVING email_count > 1
ORDER BY email_count DESC;
"""
print_results(query, run_query(query))

('zqian@umich.edu', 242)
('cwen@iupui.edu', 176)
('mmmay@indiana.edu', 161)
('ian@caret.cam.ac.uk', 127)
('chmaurer@iupui.edu', 116)
('aaronz@vt.edu', 112)
('jimeng@umich.edu', 105)
('rjlowe@iupui.edu', 100)
('david.horwitz@uct.ac.za', 85)
('dlhaines@umich.edu', 84)
('wagnermr@iupui.edu', 59)
('gjthomas@iupui.edu', 51)
('ajpoland@iupui.edu', 48)
('bkirschn@umich.edu', 48)
('josrodri@iupui.edu', 46)
('sgithens@caret.cam.ac.uk', 44)
('gsilver@umich.edu', 36)
('ray@media.berkeley.edu', 33)
('nuno@ufp.pt', 30)
('stephen.marquard@uct.ac.za', 29)
('louis@media.berkeley.edu', 28)
('arwhyte@umich.edu', 27)
('gopal.ramasammycook@gmail.com', 25)
('csev@umich.edu', 19)
('zach.thomas@txstate.edu', 19)
('antranig@caret.cam.ac.uk', 18)
('kimsooil@bu.edu', 18)
('ktsao@stanford.edu', 18)
('mbreuker@loi.nl', 18)
('ostermmg@whitman.edu', 17)
('stuart.freeman@et.gatech.edu', 17)
('dsobiera@indiana.edu', 16)
('a.fish@lancaster.ac.uk', 15)
('john.ellis@rsmart.com', 14)
('hu2@iupui.edu', 10)
('jzaremba@unic

#### Question_3: What are the top 3 days with the most emails?

In [152]:
query = """
SELECT DATE(date) AS email_date, COUNT(*) AS email_count
FROM user_history
GROUP BY email_date
ORDER BY email_count DESC
LIMIT 3;
"""
print_results(query, run_query(query))

('2007-10-29', 100)
('2007-11-06', 77)
('2007-11-20', 72)


#### Question_4: How many unique domains are present in the dataset?

In [155]:
query = """
SELECT COUNT(DISTINCT SUBSTR(email, INSTR(email, '@') + 1)) AS unique_domains
FROM user_history;
"""
print_results(query, run_query(query))

(23,)


#### Question_5: Which email addresses were active on a specific date (e.g., '2025-04-12')?

In [158]:
query = """
SELECT DISTINCT email
FROM user_history
WHERE DATE(date) = '2024-04-12'
ORDER BY email;
"""
print_results(query, run_query(query))

#### Question_6: What is the average number of emails per email address?

In [161]:
query = """
SELECT AVG(email_count) AS avg_emails
FROM (
    SELECT email, COUNT(*) AS email_count
    FROM user_history
    GROUP BY email
) AS subquery;
"""
print_results(query, run_query(query))

(40.21153846153846,)


#### Question_7: Which domains sent emails on multiple days?

In [164]:
query = """
SELECT SUBSTR(email, INSTR(email, '@') + 1) AS domain
FROM user_history
GROUP BY domain
HAVING COUNT(DISTINCT DATE(date)) > 1
ORDER BY domain;
"""
print_results(query, run_query(query))

('asu.edu',)
('bu.edu',)
('caret.cam.ac.uk',)
('et.gatech.edu',)
('gmail.com',)
('indiana.edu',)
('iupui.edu',)
('lancaster.ac.uk',)
('loi.nl',)
('media.berkeley.edu',)
('rsmart.com',)
('stanford.edu',)
('txstate.edu',)
('uct.ac.za',)
('ufp.pt',)
('umich.edu',)
('unicon.net',)
('vt.edu',)
('whitman.edu',)


#### Question_8: What is the time gap (in days) between the first and last email for each email address?

In [167]:
query = """
SELECT email,
       JULIANDAY(MAX(date)) - JULIANDAY(MIN(date)) AS days_between
FROM user_history
GROUP BY email
HAVING days_between > 0
ORDER BY days_between DESC;
"""
print_results(query, run_query(query))

('ray@media.berkeley.edu', 261.1157060181722)
('zqian@umich.edu', 206.1396296294406)
('gsilver@umich.edu', 199.10818287031725)
('wagnermr@iupui.edu', 192.0918634259142)
('bkirschn@umich.edu', 190.972500000149)
('josrodri@iupui.edu', 185.299513889011)
('ian@caret.cam.ac.uk', 168.52099537057802)
('rjlowe@iupui.edu', 127.24398148152977)
('joshua.ryan@asu.edu', 125.34405092569068)
('kimsooil@bu.edu', 121.85092592611909)
('ktsao@stanford.edu', 105.14339120360091)
('david.horwitz@uct.ac.za', 100.31872685207054)
('cwen@iupui.edu', 76.83787037059665)
('chmaurer@iupui.edu', 76.21078703738749)
('aaronz@vt.edu', 75.83212962979451)
('louis@media.berkeley.edu', 71.27917824080214)
('mmmay@indiana.edu', 65.93167824111879)
('dlhaines@umich.edu', 64.12526620412245)
('jimeng@umich.edu', 62.66005787020549)
('zach.thomas@txstate.edu', 61.38085648138076)
('wang58@iupui.edu', 61.09478009259328)
('ajpoland@iupui.edu', 60.264907407574356)
('gopal.ramasammycook@gmail.com', 58.99780092621222)
('antranig@caret.c

#### Question_9: Which email addresses sent emails in the morning (before 12 PM)?

In [170]:
query = """
SELECT DISTINCT email
FROM user_history
WHERE STRFTIME('%H', date) < '12'
ORDER BY email;
"""
print_results(query, run_query(query))

('a.fish@lancaster.ac.uk',)
('aaronz@vt.edu',)
('ajpoland@iupui.edu',)
('antranig@caret.cam.ac.uk',)
('arwhyte@umich.edu',)
('bahollad@indiana.edu',)
('bkirschn@umich.edu',)
('chmaurer@iupui.edu',)
('csev@umich.edu',)
('cwen@iupui.edu',)
('david.horwitz@uct.ac.za',)
('dlhaines@umich.edu',)
('ggolden@umich.edu',)
('gjthomas@iupui.edu',)
('gopal.ramasammycook@gmail.com',)
('gsilver@umich.edu',)
('hu2@iupui.edu',)
('ian@caret.cam.ac.uk',)
('jimeng@umich.edu',)
('jleasia@umich.edu',)
('john.ellis@rsmart.com',)
('joshua.ryan@asu.edu',)
('josrodri@iupui.edu',)
('jzaremba@unicon.net',)
('kimsooil@bu.edu',)
('ktsao@stanford.edu',)
('lance@indiana.edu',)
('louis@media.berkeley.edu',)
('mbreuker@loi.nl',)
('mmmay@indiana.edu',)
('nuno@ufp.pt',)
('ray@media.berkeley.edu',)
('rjlowe@iupui.edu',)
('sgithens@caret.cam.ac.uk',)
('ssmail@indiana.edu',)
('stephen.marquard@uct.ac.za',)
('stuart.freeman@et.gatech.edu',)
('tnguyen@iupui.edu',)
('wagnermr@iupui.edu',)
('wang58@iupui.edu',)
('zach.thomas@tx

#### Question_10: What is the total number of emails in the database?

In [173]:
query = "SELECT COUNT(*) AS total_emails FROM user_history;"
print_results(query, run_query(query))

(2091,)
